calculate the Intra-TAD_frequency in each condition

## goal

ChrInt_boxplot

In [4]:
import numpy as np
import pandas as pd
import hicstraw
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib import backends
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib import rcParams
import os
import re
import scipy

from statannot import add_stat_annotation
from statannotations.Annotator import Annotator

In [5]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
print(os.getcwd())
os.chdir('/content/drive/MyDrive/ColabNotebooks/')
print(os.getcwd())

/content
/content/drive/MyDrive/ColabNotebooks


## take a look at  hic data

In [ ]:

hic_file = "H_hicpro2juicebox/HL60_Ctrl.allValidPairs.hic"
hic_test = hicstraw.HiCFile(hic_file)



In [9]:
print(hic_test.getGenomeID())
print(hic_test.getResolutions())


/HiC-Pro_3.0.0/annotation/chrom_hg19.sizes
[2500000, 1000000, 500000, 250000, 100000, 50000, 25000, 10000, 5000]


####

In [10]:
chrom_length = {}
for chrom in hic_test.getChromosomes():
    # print(chrom.name, chrom.length)
    chrom_length[chrom.name] = chrom.length
chrom_length


{'ALL': 3095693,
 '1': 249250621,
 '2': 243199373,
 '3': 198022430,
 '4': 191154276,
 '5': 180915260,
 '6': 171115067,
 '7': 159138663,
 '8': 146364022,
 '9': 141213431,
 '10': 135534747,
 '11': 135006516,
 '12': 133851895,
 '13': 115169878,
 '14': 107349540,
 '15': 102531392,
 '16': 90354753,
 '17': 81195210,
 '18': 78077248,
 '19': 59128983,
 '20': 63025520,
 '21': 48129895,
 '22': 51304566,
 'X': 155270560,
 'Y': 59373566,
 'M': 16571}

The "All" chromosome is a zoomed out all-by-all view that is used for Juicebox visualization. The remaining chromosomes are the actual ones used for any analysis.

In [11]:
## based on the above output, it maybe different

del chrom_length["ALL"]
# del chrom_length["All"]
del chrom_length["M"]
# del chrom_length["chrM"]
# del chrom_length[""]
# del chrom_length["Y"]
# chrom_length

In [ ]:

def All_ChrInt_in_a_TAD(anchor, hic_getMatrixZoomData_dict, chrom_length, norm, resolution):

    # contact = {}
    contact = []
    ## step 1: go through every anchors and locate the bin

    for i in range(anchor.shape[0]):
        # two anchor in the same chr
        Chr = anchor.iloc[i,0]

        Start1 =  anchor.iloc[i,1]
        End1 = anchor.iloc[i,2]

        Start2 =  anchor.iloc[i,4]
        End2 = anchor.iloc[i,5]
    ## step 2: get the small matrix of the anchor

        # matrix_chr = hic.getMatrixZoomData( Chr[3:], Chr[3:], "oe", norm, "BP", resolution)
        matrix_chr = hic_getMatrixZoomData_dict[Chr]

        Start1_bin = int(Start1/resolution)*resolution
        End1_bin = int(End1/resolution)*resolution

        Start2_bin = int(Start2/resolution)*resolution
        End2_bin = int(End2/resolution)*resolution

        numpy_matrix_anchor = matrix_chr.getRecordsAsMatrix(Start1_bin, End1_bin ,Start2_bin, End2_bin)


    ## step 3: turn the symmetric matrix to arrary & save the data to the dict

        contact.append(np.sum(numpy_matrix_anchor))


    print("Number of TAD we are ploting: "+str(len(contact)))


    return  contact

## run on the data

In [29]:
# here DO NOT add \ in My Drive


hic_path = 'H_hicpro2juicebox/'
# ! ls H_hicpro2juicebox | grep -v "rep" | grep ".allValidPairs.hic"

In [30]:
TAD_as_hub_file_path = 'GENOVA_TADs/'
# TAD_as_hub_file_path = '/home/qchen/PhD/ExhaustedT/hichub_asso_CTCF_ATAC/'


In [31]:
! ls $TAD_as_hub_file_path | grep -v "as_hub_format.csv"

IntraTAD_ChrInt.pdf
TAD_Ctrl_10Kb.bed
TAD_RA2d_10Kb.bed
TAD_RA4d_10Kb.bed
TAD_RA4h_10Kb.bed


In [ ]:


norm = "KR"
resolution = 10**4


## calculate inTAD ChrInt in 4 conditions (within respectively TAD )

## hic files

In [37]:
# ls /home/qchen/PhD/ExhaustedT/Hic/results/hic_hicpro2juicebox |grep -v "dist"|grep "m.hic"

! ls H_hicpro2juicebox | grep -v "rep" | grep ".allValidPairs.hic"

HL60_2d.allValidPairs.hic
HL60_4d.allValidPairs.hic
HL60_4h.allValidPairs.hic
HL60_Ctrl.allValidPairs.hic


In [38]:
Hic_files = [f for f in os.listdir(hic_path) if re.match(r'.*.allValidPairs.hic', f)]
Hic_files

['HL60_4d.allValidPairs.hic',
 'HL60_4h.allValidPairs.hic',
 'HL60_2d.allValidPairs.hic',
 'HL60_Ctrl.allValidPairs.hic']

In [ ]:

myorder = [3,1,2,0]

Hic_files = [Hic_files[i] for i in myorder]
Hic_files

['HL60_Ctrl.allValidPairs.hic',
 'HL60_4h.allValidPairs.hic',
 'HL60_2d.allValidPairs.hic',
 'HL60_4d.allValidPairs.hic']

In [42]:
hic_path +  Hic_files[0]

'H_hicpro2juicebox/HL60_Ctrl.allValidPairs.hic'

In [43]:
H = "HL60_Ctrl.allValidPairs.hic"

In [44]:
H.split('.')[0]

'HL60_Ctrl'

In [45]:
Hic_files

['HL60_Ctrl.allValidPairs.hic',
 'HL60_4h.allValidPairs.hic',
 'HL60_2d.allValidPairs.hic',
 'HL60_4d.allValidPairs.hic']

In [46]:
for i in range(len(Hic_files)):
    print(i)
    print(Hic_files[i])

0
HL60_Ctrl.allValidPairs.hic
1
HL60_4h.allValidPairs.hic
2
HL60_2d.allValidPairs.hic
3
HL60_4d.allValidPairs.hic


## TAD files

In [47]:
TAD_Files = [f for f in sorted(os.listdir(TAD_as_hub_file_path)) if re.match(r'.*.as_hub_format.csv', f)]
TAD_Files

['TAD_Ctrl_10Kb_as_hub_format.csv',
 'TAD_RA2d_10Kb_as_hub_format.csv',
 'TAD_RA4d_10Kb_as_hub_format.csv',
 'TAD_RA4h_10Kb_as_hub_format.csv']

In [48]:
Conditions= [x.split('_as_hub_format.csv')[0] for x in TAD_Files]
Conditions

['TAD_Ctrl_10Kb', 'TAD_RA2d_10Kb', 'TAD_RA4d_10Kb', 'TAD_RA4h_10Kb']

In [49]:
TAD_dict = {}


for p in Conditions:
    TAD_dict[p] = pd.read_csv(TAD_as_hub_file_path + p +"_as_hub_format.csv",header = 0 )

In [50]:
[TAD_dict[p].shape for p in TAD_dict.keys()]

[(3885, 6), (3853, 6), (3828, 6), (4009, 6)]

In [51]:
list(TAD_dict.keys())

['TAD_Ctrl_10Kb', 'TAD_RA2d_10Kb', 'TAD_RA4d_10Kb', 'TAD_RA4h_10Kb']

In [52]:
TAD_dict[list(TAD_dict.keys())[0]].head()

,chr1,left_Start,left_End,chr2,right_Start,right_End
0,chr1,1240000,1630000,chr1,1240000,1630000
1,chr1,1630000,1850000,chr1,1630000,1850000
2,chr1,1850000,2020000,chr1,1850000,2020000
3,chr1,2020000,2430000,chr1,2020000,2430000
4,chr1,2430000,3340000,chr1,2430000,3340000


## calculate inTAD interactions

In [53]:
Hic_files

['HL60_Ctrl.allValidPairs.hic',
 'HL60_4h.allValidPairs.hic',
 'HL60_2d.allValidPairs.hic',
 'HL60_4d.allValidPairs.hic']

In [54]:
Conditions

['TAD_Ctrl_10Kb', 'TAD_RA2d_10Kb', 'TAD_RA4d_10Kb', 'TAD_RA4h_10Kb']

In [55]:
list(TAD_dict.keys())

['TAD_Ctrl_10Kb', 'TAD_RA2d_10Kb', 'TAD_RA4d_10Kb', 'TAD_RA4h_10Kb']

In [ ]:
### take a very long time. becauae the function I defined is not efficient

Hic_contact_list = []

for i in range(4):
    print(i)
    H = Hic_files[i]
    print(H)

    print(list(TAD_dict.keys())[i])

    anchor =TAD_dict[list(TAD_dict.keys())[i]]

    print(H)
    # 1. for each hic file, calculated the gene asso hub ChrInt
    print(H.split('.')[0])

    #  ---> load corresponding hic data
    # hic = hicstraw.HiCFile(hic_path + H )
    hic_getMatrixZoomData_dict = hic_getMatrixZoomData(hic_path + H)

    # calculate the hub interaction in the hic file
    # Contact = Tri_hub_mean_Interaction(anchor, hic, chrom_length, norm, resolution)

    Hic_contact_list.append(All_ChrInt_in_a_TAD(anchor, hic_getMatrixZoomData_dict, chrom_length, norm, resolution))

    # print()

0
HL60_Ctrl.allValidPairs.hic
TAD_Ctrl_10Kb
HL60_Ctrl.allValidPairs.hic
HL60_Ctrl
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
Number of TAD we are ploting: 3885
1
HL60_4h.allValidPairs.hic
TAD_RA2d_10Kb
HL60_4h.allValidPairs.hic
HL60_4h
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
Number of TAD we are ploting: 3853
2
HL60_2d.allValidPairs.hic
TAD_RA4d_10Kb
HL60_2d.allValidPairs.hic
HL60_2d
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
Number of TAD we are ploting: 3828
3
HL60_4d.allValidPairs.hic
TAD_RA4h_10Kb
HL60_4d.allValidPairs.hic
HL60_4d
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
Number of TAD we are ploting: 4009


In [58]:
len(Hic_contact_list)

4

In [59]:
[len(p) for p in Hic_contact_list]

[3885, 3853, 3828, 4009]

In [60]:
min([len(p) for p in Hic_contact_list])

3828


# plot


In [61]:
matplotlib

Using matplotlib backend: <object object at 0x79f8d81bbac0>


What is the difference between the Wilcoxon Rank Sum Test and the Mann-Whitney U test?

The Mann–Whitney U test / Wilcoxon rank-sum test is not the same as the Wilcoxon signed-rank test, although both are nonparametric and involve summation of ranks.

The Mann–Whitney U test is applied to independent samples.

The Wilcoxon signed-rank test is applied to matched or dependent samples.



Wilcoxon rank-sum test is used to compare two independent samples, while Wilcoxon signed-rank test is used to compare two related samples, matched samples, or to conduct a paired difference test of repeated measurements on a single sample to assess whether their population mean ranks differ.


In [62]:
    tmp = pd.DataFrame([ pd.Series(value) for value in Hic_contact_list ])

    df = tmp.transpose()

    # df = pd.DataFrame({"0h" : Hic_contact_list[0],
    #                  "4h" : Hic_contact_list[1],
    #                  "2d" : Hic_contact_list[2],
    #                   "4d" : Hic_contact_list[3]})
    df = df.rename(columns={0:"0h", 1:"4h", 2:"2d",3: "4d"})
    df = df.iloc[0:min([len(p) for p in Hic_contact_list]),]
    df

,0h,4h,2d,4d
0,10091.248120,9720.097024,0.000000,3985.439797
1,4537.259329,9013.843748,31846.721340,5694.623615
2,2819.043336,3666.904326,22760.022560,3134.384739
3,8748.647559,49761.896664,3637.582550,16700.777145
4,20364.144940,52337.240274,4817.454499,15758.762617
...,...,...,...,...
3823,4.194928,5819.972595,2483.671453,5539.082718
3824,5787.070444,6758.804312,4782.834680,3899.267013
3825,5953.560309,5398.477129,11741.870874,20250.999613
3826,3429.398282,10236.395885,5310.413136,11511.293582


In [ ]:


    from matplotlib.backends.backend_pdf import PdfPages
    pdf = PdfPages("GENOVA_TADs/IntraTAD_ChrInt_KR.pdf")


    tmp = pd.DataFrame([ pd.Series(value) for value in Hic_contact_list ])

    df = tmp.transpose()

    df = df.rename(columns={0:"0h", 1:"4h", 2:"2d",3: "4d"})
    # when plot need to have the same length
    df = df.iloc[0:min([len(p) for p in Hic_contact_list]),]
    df.to_csv("./TAD_region_ChrInt.csv")

    df2 = pd.melt(df,var_name='Condition', value_name='ChrInt')

    # draw boxplot

    ## chnage boxplot color:
    PROPS = {
        'boxprops':{'facecolor':'none', 'edgecolor':'black'},
        'medianprops':{'color':'black'},
        'whiskerprops':{'color':'black', 'linestyle':'--'},
        'capprops':{'color':'black'}
    }
    # 1. horizontal ---- but hard to add p value
    plt.figure(figsize=(6,4.5))
    ax1 = sns.boxplot(x='ChrInt',y='Condition', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    sns.stripplot(x='ChrInt',y='Condition', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax1 )

    ax1.set_title("Intra-TAD ChrInt")


    # ---------Remove the top and right spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

    # plt.show()

    # pvalue1 = scipy.stats.mannwhitneyu(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "Stem"]["ChrInt"].dropna())
    pvalue1 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "4h"]["ChrInt"].dropna())
    print( "0h vs 4h "  + str( pvalue1))

    # pvalue2 = scipy.stats.mannwhitneyu(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "Term"]["ChrInt"].dropna())
    pvalue2 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "2d"]["ChrInt"].dropna())
    print( "0h vs 2d  "  + str( pvalue2))

    pvalue3 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "4d"]["ChrInt"].dropna())
    print( "0h vs 4d  "  + str( pvalue2))

    pdf.savefig()


    # 2. Vertical ----easy to add p value
    plt.figure(figsize=(4.5,6))

    ax2 = sns.boxplot(x='Condition', y='ChrInt', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    sns.stripplot(x='Condition', y='ChrInt', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax2)


    ax2.set_title("Intra-TAD ChrInt")
    # MOVE Y AXIS TO THE RIGHT
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    # ---------Remove the top andLEFT spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=False, left= True, bottom=False, offset=None, trim=False)

    from statannot import add_stat_annotation


    add_stat_annotation(ax2, data=df2,x='ChrInt',y='Condition',
                        box_pairs=[("0h", "4h"), ("0h", "2d"),("0h", "4d")],
                        test='Wilcoxon', text_format='full', loc='inside')

    pdf.savefig()

    # 3. anova test
    plt.figure(figsize=(4.5,6))

    ax2 = sns.boxplot(x='Condition', y='ChrInt', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    sns.stripplot(x='Condition', y='ChrInt', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax2)

    # ax2.set_title(Gene)
    # MOVE Y AXIS TO THE RIGHT
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    # ---------Remove the top andLEFT spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=False, left= True, bottom=False, offset=None, trim=False)

    # stats f_oneway functions takes the groups as input and returns ANOVA F and p value
    import scipy.stats as stats
    fvalue, pvalue = stats.f_oneway(df['0h'], df['4h'], df['2d'], df['4d'])
    print(fvalue, pvalue)
    significance = "*" if pvalue < 0.05 else "ns"

    T= "Intra-TAD ChrInt"+" \n"+ f'ANOVA p-value: {pvalue:.4f} {significance}'
    ax2.set_title(T)

    pdf.savefig()


    pdf.close()

0h vs 4h RanksumsResult(statistic=-17.205239893549532, pvalue=2.425701652351121e-66)
0h vs 2d  RanksumsResult(statistic=-10.74584188049242, pvalue=6.199374102684813e-27)
0h vs 4d  RanksumsResult(statistic=-10.74584188049242, pvalue=6.199374102684813e-27)
Using zero_method  wilcox
0h v.s. 4h: Wilcoxon test (paired samples) with Bonferroni correction, P_val=2.050e-53 stat=2.607e+06
Using zero_method  wilcox
0h v.s. 2d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=1.873e-22 stat=2.990e+06
Using zero_method  wilcox
0h v.s. 4d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=8.149e-21 stat=3.017e+06
63.54687406140714 7.947101286604348e-41


In [ ]:


    from matplotlib.backends.backend_pdf import PdfPages
    pdf = PdfPages("GENOVA_TADs/IntraTAD_ChrInt_KR_2.pdf")


    tmp = pd.DataFrame([ pd.Series(value) for value in Hic_contact_list ])

    df = tmp.transpose()

    df = df.rename(columns={0:"0h", 1:"4h", 2:"2d",3: "4d"})
    # when plot need to have the same length
    df = df.iloc[0:min([len(p) for p in Hic_contact_list]),]
    df.to_csv("./TAD_region_ChrInt.csv")

    df2 = pd.melt(df,var_name='Condition', value_name='ChrInt')

    # draw boxplot

    ## chnage boxplot color:
    PROPS = {
        'boxprops':{'facecolor':'none', 'edgecolor':'black'},
        'medianprops':{'color':'black'},
        'whiskerprops':{'color':'black', 'linestyle':'--'},
        'capprops':{'color':'black'}
    }
    # 1. horizontal ---- but hard to add p value
    plt.figure(figsize=(6,4.5))
    ax1 = sns.boxplot(x='ChrInt',y='Condition', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    # sns.stripplot(x='ChrInt',y='Condition', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax1 )

    ax1.set_title("Intra-TAD ChrInt")


    # ---------Remove the top and right spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

    # plt.show()

    # pvalue1 = scipy.stats.mannwhitneyu(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "Stem"]["ChrInt"].dropna())
    pvalue1 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "4h"]["ChrInt"].dropna())
    print( "0h vs 4h "  + str( pvalue1))

    # pvalue2 = scipy.stats.mannwhitneyu(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "Term"]["ChrInt"].dropna())
    pvalue2 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "2d"]["ChrInt"].dropna())
    print( "0h vs 2d  "  + str( pvalue2))

    pvalue3 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "4d"]["ChrInt"].dropna())
    print( "0h vs 4d  "  + str( pvalue2))

    pdf.savefig()


    # 2. Vertical ----easy to add p value
    plt.figure(figsize=(4.5,6))

    ax2 = sns.boxplot(x='Condition', y='ChrInt', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    # sns.stripplot(x='Condition', y='ChrInt', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax2)


    ax2.set_title("Intra-TAD ChrInt")
    # MOVE Y AXIS TO THE RIGHT
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    # ---------Remove the top andLEFT spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=False, left= True, bottom=False, offset=None, trim=False)

    from statannot import add_stat_annotation

    add_stat_annotation(ax2, data=df2,x='ChrInt',y='Condition',
                        box_pairs=[("0h", "4h"), ("0h", "2d"),("0h", "4d")],
                        test='Wilcoxon', text_format='full', loc='inside')

    pdf.savefig()

    # 3. anova test
    plt.figure(figsize=(4.5,6))

    ax2 = sns.boxplot(x='Condition', y='ChrInt', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    # sns.stripplot(x='Condition', y='ChrInt', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax2)

    # ax2.set_title(Gene)
    # MOVE Y AXIS TO THE RIGHT
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    # ---------Remove the top andLEFT spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=False, left= True, bottom=False, offset=None, trim=False)

    # stats f_oneway functions takes the groups as input and returns ANOVA F and p value
    import scipy.stats as stats
    fvalue, pvalue = stats.f_oneway(df['0h'], df['4h'], df['2d'], df['4d'])
    print(fvalue, pvalue)
    significance = "*" if pvalue < 0.05 else "ns"

    T= "Intra-TAD ChrInt"+" \n"+ f'ANOVA p-value: {pvalue:.4f} {significance}'
    ax2.set_title(T)

    pdf.savefig()


    pdf.close()

0h vs 4h RanksumsResult(statistic=-17.205239893549532, pvalue=2.425701652351121e-66)
0h vs 2d  RanksumsResult(statistic=-10.74584188049242, pvalue=6.199374102684813e-27)
0h vs 4d  RanksumsResult(statistic=-10.74584188049242, pvalue=6.199374102684813e-27)
Using zero_method  wilcox
0h v.s. 4h: Wilcoxon test (paired samples) with Bonferroni correction, P_val=2.050e-53 stat=2.607e+06
Using zero_method  wilcox
0h v.s. 2d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=1.873e-22 stat=2.990e+06
Using zero_method  wilcox
0h v.s. 4d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=8.149e-21 stat=3.017e+06
63.54687406140714 7.947101286604348e-41
